(dolly_v2_deepspeed_instruction_finetune)=

# Dolly-V2-3B Instruction Fine-Tuning wiht Ray AIR and DeepSpeed

In this demonstration, we'll show how to use the Ray AIR for Dolly V2 3B model instruction fine-tuning using the deep-speed framework. Please uncomment the next two cells and install the following libraries dependencies.

This work builds upon [existing efforts](https://github.com/ray-project/ray/blob/master/doc/source/ray-air/examples/gptj_deepspeed_fine_tuning.ipynb) by incorporating an instruction fine-tuning component.

In [4]:
!conda install -c conda-forge mpi4py -y
!conda install gcc gxx_linux-64 -y

done
Solving environment: - 
  - conda-forge/linux-64::mpi-1.0-openmpi, conda-forge/linux-64::openmpi-4.0.4-hdf1f1ad_0, defaults/linux-64::mpi4py-3.1.4-py310h3e5f7c9_0
  - conda-forge/linux-64::mpi-1.0-mpich, defaults/linux-64::mpi4py-3.1.4-py310hfc96bbd_0, defaults/linux-64::mpich-3.3.2-hc856adbdone


==> WARNING: A newer version of conda exists. <==
  current version: 23.3.1
  latest version: 23.7.4

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.7.4



## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - mpi4py


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2023.7.22  |       hbcca054_0         146 KB  conda-forge
    certifi-2023.7.22          |     pyhd8ed1ab_0         150 KB  conda-forge
    conda-23.7

In [1]:
! pip install "ray==2.5.1" "accelerate==0.16.0" "datasets==2.12.0" "transformers==4.26.0"  "torch==1.13.0" "deepspeed==0.9.2"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.2/56.2 MB 3.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 63.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 890.1/890.1 MB 5.2 MB/s eta 0:00:0000:0100:03
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.3/779.3 kB 42.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 530.8/530.8 kB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 47.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 8.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

In [6]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 2.8 MB/s eta 0:00:00


In [1]:
import numpy as np
import pandas as pd
import os
from transformers import AutoTokenizer
import ray.data
import ray
from datasets import load_dataset
import evaluate
from transformers import Trainer, TrainingArguments
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
)
from transformers.utils.logging import enable_progress_bar
import torch

import transformers
from ray.train.huggingface import TransformersTrainer
from ray.air.config import ScalingConfig

## Set up Ray <a name="setup"></a>

First, we will use 2 workers, each being assigned 1 GPU and 28 CPUs.

In [2]:
model_name = "databricks/dolly-v2-3b"
use_gpu = True
num_workers = 1
cpus_per_worker = 12

In [3]:
#fq_ray_ip = #<replace-this-with-your-ray-server-ip-address>

In [4]:
pip_env = {
    "pip": [
        "datasets==2.12.0",
        "evaluate==0.4.0",
        "accelerate==0.16.0",  # https://github.com/OpenGVLab/InternImage/issues/111
        "transformers==4.26.0",
        "torch==1.13.0",
        "deepspeed==0.9.2",
        "ipython==8.14.0",
    ]
}

In [5]:
conda_env = {
    "conda": {
        "dependencies": ["mpi4py", "pip", pip_env]
    }  # pip install mpi4py won't work, use conda install instead
}

In [6]:
# ray.init(
#     f"ray://{fq_ray_ip}:10001",  # Note: the port and ip-address depends on your ray server setup.
#     runtime_env=conda_env,
# )

## Loading the dataset <a name="load"></a>

We will be fine-tuning the model on the [`alpaca-cleaned` dataset](https://datasets-server.huggingface.co/splits?dataset=yahma%2Falpaca-cleaned), comprised of 51,000 lines of Q&A. The aim will be to make the databricks model better at generating answer by following the instruction.

We will use `generate_prompt` function to prepare our dataset for instruction fine-tuning.

In [7]:
current_dataset = load_dataset("yahma/alpaca-cleaned")
current_dataset

Found cached dataset json (/root/.cache/huggingface/datasets/yahma___json/yahma--alpaca-cleaned-5d24553f76c14acc/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['instruction', 'output', 'input'],
        num_rows: 51760
    })
})

In [8]:
def generate_prompt(data_point):
    # ref: https://github.com/tloen/alpaca-lora
    if data_point["instruction"]:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{data_point["instruction"]}

### Input:
{data_point["input"]}

### Response:
{data_point["output"]}"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{data_point["instruction"]}

### Response:
{data_point["output"]}"""

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side="left")
tokenizer.pad_token_id = 0
CUTOFF_LEN = 128

current_dataset = current_dataset.shuffle().map(
    lambda data_point: tokenizer(
        generate_prompt(data_point),
        truncation=True,
        max_length=CUTOFF_LEN,
        padding="max_length",
    ),
)

Map:   0%|          | 0/51760 [00:00<?, ? examples/s]

In [10]:
ray_datasets = ray.data.from_huggingface(current_dataset["train"])
ray_datasets

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


2023-09-21 19:37:15,152	WARNING utils.py:593 -- Detecting docker specified CPUs. In previous versions of Ray, CPU detection in containers was incorrect. Please ensure that Ray has enough CPUs allocated. As a temporary workaround to revert to the prior behavior, set `RAY_USE_MULTIPROCESSING_CPU_COUNT=1` as an env var before starting Ray. Set the env var: `RAY_DISABLE_DOCKER_CPU_WARNING=1` to mute this warning.
2023-09-21 19:37:15,154	WARNING utils.py:605 -- Ray currently does not support initializing Raywith fractional cpus. Your num_cpus will be truncated from 15.36 to 15.
2023-09-21 19:37:15,232	INFO worker.py:1636 -- Started a local Ray instance.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


2023-09-21 19:37:20,765	WARNING dataset.py:253 -- Important: Ray Data requires schemas for all datasets in Ray 2.5. This means that standalone Python objects are no longer supported. In addition, the default batch format is fixed to NumPy. To revert to legacy behavior temporarily, set the environment variable RAY_DATA_STRICT_MODE=0 on all cluster processes.

Learn more here: https://docs.ray.io/en/master/data/faq.html#migrating-to-strict-mode


MaterializedDataset(
   num_blocks=1,
   num_rows=51760,
   schema={
      instruction: string,
      output: string,
      input: string,
      input_ids: list<item: int32>,
      attention_mask: list<item: int8>
   }
)

## Instruction fine-tuning the model with Ray AIR

In [11]:
def trainer_init_per_worker(train_dataset, eval_dataset=None, **config):
    batch_size = config.get("batch_size", 1)
    epochs = config.get("epochs", 1)
    warmup_steps = config.get("warmup_steps", 0)
    learning_rate = config.get("learning_rate", 0.00002)
    weight_decay = config.get("weight_decay", 0.01)

    deepspeed = {
        "fp16": {
            "enabled": "auto",
            "initial_scale_power": 8,
        },
        "optimizer": {
            "type": "AdamW",
            "params": {
                "lr": "auto",
                "betas": "auto",
                "eps": "auto",
            },
        },
        "zero_optimization": {
            "stage": 3,
            "offload_optimizer": {
                "device": "cpu",
                "pin_memory": True,
            },
            "offload_param": {
                "device": "cpu",
                "pin_memory": True,
            },
            "overlap_comm": True,
            "contiguous_gradients": True,
            "reduce_bucket_size": "auto",
            "stage3_prefetch_bucket_size": "auto",
            "stage3_param_persistence_threshold": "auto",
            "gather_16bit_weights_on_model_save": True,
            "round_robin_gradients": True,
        },
        "gradient_accumulation_steps": "auto",
        "gradient_clipping": "auto",
        "steps_per_print": 10,
        "train_batch_size": "auto",
        "train_micro_batch_size_per_gpu": "auto",
        "wall_clock_breakdown": False,
    }

    print(f"batch_size: {batch_size}")
    print("Preparing training arguments")
    training_args = TrainingArguments(
        output_dir="deepspeed-dolly",
        per_device_train_batch_size=batch_size,
        logging_steps=1,
        learning_rate=learning_rate,
        weight_decay=weight_decay,
        warmup_steps=warmup_steps,
        num_train_epochs=epochs,
        push_to_hub=False,
        disable_tqdm=False,
        fp16=True,
        gradient_accumulation_steps=16,
        deepspeed=deepspeed,
    )

    tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side="left")
    tokenizer.pad_token_id = 0

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float16,
    )

    enable_progress_bar()

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        tokenizer=tokenizer,
        data_collator=transformers.DataCollatorForLanguageModeling(
            tokenizer, mlm=False
        ),
    )
    return trainer

In [12]:
trainer = TransformersTrainer(
    trainer_init_per_worker=trainer_init_per_worker,
    trainer_init_config={
        "batch_size": 16,  # batch_size per device
        "epochs": 1,
    },
    scaling_config=ScalingConfig(
        num_workers=num_workers,
        use_gpu=use_gpu,
        resources_per_worker={
            "GPU": 1,
            "CPU": cpus_per_worker,
        },  # NOTE: huggingface transformers only support 1 GPU per worker.
    ),
    run_config=ray.air.RunConfig(
        sync_config=ray.tune.syncer.SyncConfig(
            sync_on_checkpoint=False  # Note: one can also set up a storage path to persist the model checkpoint to a cloud bucket
        )
    ),
    datasets={
        "train": ray_datasets,
    },
)

Finally, we call the `~ray.train.huggingface.TransformersTrainer.fit` method to start training with Ray AIR.

In [13]:
results = trainer.fit()

/opt/conda/lib/python3.10/site-packages/ray/train/base_trainer.py:581: UserWarning: Executing `.fit()` may leave less than 20% of CPUs in this cluster for Dataset execution, which can lead to resource contention or hangs. To avoid this, reserve at least 20% of node CPUs for Dataset execution by setting `_max_cpu_fraction_per_node = 0.8` in the Trainer scaling_config. See https://docs.ray.io/en/master/data/dataset-internals.html#datasets-and-tune for more info.
  tuner = Tuner(
2023-09-21 19:37:50,755	INFO tensorboardx.py:178 -- pip install "ray[tune]" to see TensorBoard files.
2023-09-21 19:37:50,756	WARNING callback.py:144 -- The TensorboardX logger cannot be instantiated because either TensorboardX or one of it's dependencies is not installed. Please make sure you have the latest version of TensorboardX installed: `pip install -U tensorboardx`


(TransformersTrainer pid=29764) 2023-09-21 19:37:56,022	WARNING dataset.py:253 -- Important: Ray Data requires schemas for all datasets in Ray 2.5. This means that standalone Python objects are no longer supported. In addition, the default batch format is fixed to NumPy. To revert to legacy behavior temporarily, set the environment variable RAY_DATA_STRICT_MODE=0 on all cluster processes.
(TransformersTrainer pid=29764) 
(TransformersTrainer pid=29764) Learn more here: https://docs.ray.io/en/master/data/faq.html#migrating-to-strict-mode
(TransformersTrainer pid=29764) 2023-09-21 19:37:57,475	INFO backend_executor.py:137 -- Starting distributed worker processes: ['29899 (172.17.0.4)']
(RayTrainWorker pid=29899) 2023-09-21 19:37:58,175	INFO config.py:86 -- Setting up process group for: env:// [rank=0, world_size=1]


(pid=29899) - RandomizeBlockOrder 1:   0%|          | 0/1 [00:00<?, ?it/s]

(pid=29899) Running 0:   0%|          | 0/1 [00:00<?, ?it/s]

(RayTrainWorker pid=29899) 2023-09-21 19:38:01,361	INFO streaming_executor.py:91 -- Executing DAG InputDataBuffer[Input] -> AllToAllOperator[RandomizeBlockOrder]
(RayTrainWorker pid=29899) 2023-09-21 19:38:01,361	INFO streaming_executor.py:92 -- Execution config: ExecutionOptions(resource_limits=ExecutionResources(cpu=None, gpu=None, object_store_memory=None), locality_with_output=False, preserve_order=False, actor_locality_enabled=True, verbose_progress=False)
(RayTrainWorker pid=29899) 2023-09-21 19:38:01,361	INFO streaming_executor.py:94 -- Tip: For detailed progress reporting, run `ray.data.DataContext.get_current().execution_options.verbose_progress = True`
(RayTrainWorker pid=29899) 2023-09-21 19:38:01,372	INFO streaming_executor.py:149 -- Shutting down <StreamingExecutor(Thread-3, stopped daemon 139699959088896)>.


...

(RayTrainWorker pid=29899) Using /root/.cache/torch_extensions/py310_cu117 as PyTorch extensions root...
(RayTrainWorker pid=29899) Creating extension directory /root/.cache/torch_extensions/py310_cu117/cpu_adam...
(RayTrainWorker pid=29899) Detected CUDA files, patching ldflags
(RayTrainWorker pid=29899) Emitting ninja build file /root/.cache/torch_extensions/py310_cu117/cpu_adam/build.ninja...
(RayTrainWorker pid=29899) Building extension module cpu_adam...
(RayTrainWorker pid=29899) Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
2023-09-21 19:39:47,693	ERROR tune_controller.py:873 -- Trial task failed for trial TransformersTrainer_59b78_00000
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/ray/air/execution/_internal/event_manager.py", line 110, in resolve_future
    result = ray.get(future)
  File "/opt/conda/lib/python3.10/site-packages/ray/_private/auto_init_hook.py", line 18, in

(RayTrainWorker pid=29899) [2/3] c++ -MMD -MF cpu_adam.o.d -DTORCH_EXTENSION_NAME=cpu_adam -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUILD_ABI=\"_cxxabi1011\" -I/opt/conda/lib/python3.10/site-packages/deepspeed/ops/csrc/includes -I/usr/local/cuda/include -isystem /opt/conda/lib/python3.10/site-packages/torch/include -isystem /opt/conda/lib/python3.10/site-packages/torch/include/torch/csrc/api/include -isystem /opt/conda/lib/python3.10/site-packages/torch/include/TH -isystem /opt/conda/lib/python3.10/site-packages/torch/include/THC -isystem /usr/local/cuda/include -isystem /opt/conda/include/python3.10 -D_GLIBCXX_USE_CXX11_ABI=0 -fPIC -std=c++14 -O3 -std=c++14 -g -Wno-reorder -L/usr/local/cuda/lib64 -lcudart -lcublas -g -march=native -fopenmp -D__AVX256__ -D__ENABLE_CUDA__ -c /opt/conda/lib/python3.10/site-packages/deepspeed/ops/csrc/adam/cpu_adam.cpp -o cpu_adam.o 
(RayTrainWorker pid=29899) ninja: build stopped: subco

Trial name,date,hostname,node_ip,pid,timestamp,trial_id
TransformersTrainer_59b78_00000,2023-09-21_19-37-56,56a026cbdb4c,172.17.0.4,29764,1695325076,59b78_00000


2023-09-21 19:39:47,705	ERROR tune.py:1107 -- Trials did not complete: [TransformersTrainer_59b78_00000]
2023-09-21 19:39:47,706	INFO tune.py:1111 -- Total run time: 116.95 seconds (116.94 seconds for the tuning loop).
2023-09-21 19:39:47,708	WARNING experiment_analysis.py:910 -- Failed to read the results for 1 trials:
- /root/ray_results/TransformersTrainer_2023-09-21_19-37-50/TransformersTrainer_59b78_00000_0_2023-09-21_19-37-50


TrainingFailedError: The Ray Train run failed. Please inspect the previous error messages for a cause. After fixing the issue (assuming that the error is not caused by your own application logic, but rather an error such as OOM), you can restart the run from scratch or continue this run.
To continue this run, you can use: `trainer = TransformersTrainer.restore("/root/ray_results/TransformersTrainer_2023-09-21_19-37-50")`.
To start a new run that will retry on training failures, set `air.RunConfig(failure_config=air.FailureConfig(max_failures))` in the Trainer's `run_config` with `max_failures > 0`, or `max_failures = -1` for unlimited retries.